In [58]:
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import ReLU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [59]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Dark-
[nltk_data]     Devil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Dark-
[nltk_data]     Devil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
#read data
data = pd.read_csv("C:\\Users\\Dark-Devil\\Desktop\\IMDB Dataset.csv")

In [61]:
#size of data
data.shape

(50000, 2)

In [62]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [63]:
#check null value 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [64]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [65]:
data['sent_bin'] = data['sentiment'].replace({'positive': 1, 'negative': 0})

In [66]:
data.head()

,review,sentiment,sent_bin
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [67]:
df = data[0:5000]

In [68]:
df.shape

(5000, 3)

In [69]:
stop_word_text = "a, an, the, and, or, but, if, then, else, when, where, who, whom, which, that, this, these, those, in, on, at, to, from, by, for, of, with, without, over, under, above, below, between, among, through, throughout, until, while, since, during, within, without, beyond, beside, between, except, but, up, down, in, out, off, above, below, under, too, very, so, such, just, as, both, neither, either, although, because, since, so that, though, unless, until, whether, while, why"

In [70]:
stop_words = stop_word_text.split(",")

In [71]:
#filtering the text
def full_form(text):
    text = text.lower()
    plain = re.sub(r'[<>?\.,!"(\)\/[\]]', '', text)
    plain = plain.replace("don't", "do not")
    plain = plain.replace("won't", "will not")
    plain = plain.replace("haven't", "have not")
    plain = plain.replace("can't", "cannot")
    plain = plain.replace("she's", "she is")
    plain = plain.replace("he's", "he is")
    plain = plain.replace("there're", "there are")
    plain = plain.replace("they'd", "they would")
    plain = plain.replace("\'ll", " will")
    return plain              

In [72]:
#initialize the stemmer and lemmentizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [73]:
#preprocessing  text 
def remove_stop_words(data):
  corpus = []
  for i in range(0, len(data)):
    #split the sentence
    plain = full_form(data['review'][i])
    sentence = plain.split()
    
    #check and stem the word
    review_processed = [stemmer.stem(word) for word in sentence if not word in stop_words]
    #rebuild the sentence
    review_joint = ' '.join(review_processed)
    #add the sentence into list
    corpus.append(review_joint)
  return corpus

In [74]:
review_processed = remove_stop_words(data)

In [75]:
y = data['sent_bin']

In [76]:
#vocabular size
voc_size = 5000

In [77]:
#convert into one hot vector
onehot_text = [one_hot(word, voc_size) for word in review_processed]

In [78]:
len(onehot_text)

50000

# Embedding

In [79]:
sent_length = 200

In [80]:
#embedding
embedd_docs = pad_sequences(onehot_text, padding='pre', maxlen=sent_length)

In [81]:
len(review_processed[0].split()), len(onehot_text[0])

(307, 307)

In [82]:
embedd_docs[0]

array([1481, 1577, 3532, 4945, 4290, 2186, 3401, 3703,  828, 2302,  178,
       3798, 2371, 2302, 3474, 4369, 4537, 2026, 2689, 3723,  480, 3150,
       2637,   50, 1656,  545, 3532, 2302,  368, 3892, 4290,   50, 2017,
       1352, 2841, 4675, 1663, 2789, 4526, 4312, 3679, 2689, 2283, 1481,
       1876, 4231, 2380,  860, 2689, 2381, 2409, 2276, 2212, 4815, 4080,
       3238, 1228, 4357, 3235, 2302, 2984, 2671,  828, 2302, 3830,   50,
       2462, 1352, 2302,  382,  515, 4377, 1651, 3798, 4578, 3830, 2558,
       2838, 1390, 4902, 2017,  168,   92,  415, 2711, 1390, 4814, 1390,
       3784, 1438, 4179, 1943, 2302, 1235, 1969, 1228,  707, 1417, 4073,
        656, 4651, 3150, 2647, 4377, 1888, 2171, 1228, 4553, 3235, 1228,
       1888, 2139,   92, 4377,  421, 4651, 1228,   90, 2986, 1228, 3084,
        237,   92, 3292, 2689, 4296, 4993, 1352, 2302,  545,  486,  828,
       1510, 3558, 1656, 1189, 3558,  421,   45, 2554, 1192, 3852,  783,
       4174,  897, 4428,   92,  851, 3009, 3852,  7

In [83]:
#shape of data
len(embedd_docs), y.shape

(50000, (50000,))

In [84]:
#convert into numpy array
X_data = np.array(embedd_docs)
y_data = np.array(y)
#y_data = np.squeeze(y_data)

In [85]:
X_data.shape, y_data.shape

((50000, 200), (50000,))

In [86]:
#split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

In [87]:
print(f'training: {X_train.shape} - {y_train.shape}')
print(f'testing: {X_test.shape} - {y_test.shape}')

training: (35000, 200) - (35000,)
testing: (15000, 200) - (15000,)


In [88]:
y_train[0:10]

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0], dtype=int64)

# Model

In [89]:
## Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [90]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
547/547 [==============================] - 129s 232ms/step - loss: 0.4110 - accuracy: 0.8111 - val_loss: 0.3506 - val_accuracy: 0.8541
Epoch 2/10
547/547 [==============================] - 155s 284ms/step - loss: 0.3002 - accuracy: 0.8754 - val_loss: 0.3175 - val_accuracy: 0.8629
Epoch 3/10
547/547 [==============================] - 149s 272ms/step - loss: 0.2499 - accuracy: 0.8977 - val_loss: 0.3395 - val_accuracy: 0.8641
Epoch 4/10
547/547 [==============================] - 151s 276ms/step - loss: 0.2250 - accuracy: 0.9091 - val_loss: 0.3387 - val_accuracy: 0.8599
Epoch 5/10
547/547 [==============================] - 145s 266ms/step - loss: 0.1902 - accuracy: 0.9245 - val_loss: 0.3760 - val_accuracy: 0.8629
Epoch 6/10
547/547 [==============================] - 149s 272ms/step - loss: 0.1627 - accuracy: 0.9373 - val_loss: 0.4135 - val_accuracy: 0.8640
Epoch 7/10
547/547 [==============================] - 151s 275ms/step - loss: 0.1363 - accuracy: 0.9503 - val_loss: 0.4076 -

# Model Evaluation

In [108]:
model.evaluate(X_test, y_test)

469/469 [==============================] - 11s 24ms/step - loss: 0.5542 - accuracy: 0.8619


[0.5541790127754211, 0.8618666529655457]

In [109]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

469/469 [==============================] - 11s 24ms/step


In [110]:
#confusion matrix 
confusion_matrix(y_test,predictions)

array([[6203, 1208],
       [ 864, 6725]], dtype=int64)

In [111]:
accuracy_score(y_test,predictions)

0.8618666666666667

# Testing model with text

In [112]:
print(data['review'][0],'\n')
print(data['sentiment'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [125]:
#preprocessing  text 
def remove_stop_words(data):
    corpus = []
 
    #split the sentence
    plain = full_form(data)
    sentence = plain.split()
    
    #check and stem the word
    review_processed = [stemmer.stem(word) for word in sentence if not word in stop_words]
    #rebuild the sentence
    review_joint = ' '.join(review_processed)
    #add the sentence into list
    corpus.append(review_joint)
    return corpus

In [130]:
text = "this movie is very good and i like this movie"
#preprocessing 
processed_text = remove_stop_words(text)
#one hot encoding
onehot_sent = [one_hot(word, voc_size) for word in processed_text]
#embedding
embedd_docs = pad_sequences(onehot_sent, padding='pre', maxlen=sent_length)
#convert into numpy array
X_sample = np.array(embedd_docs)

In [131]:
processed_text

['thi movi is veri good and i like thi movi']

In [147]:
sample_prediction = (rnn.predict(X_sample) > 0.5).astype("int32")

1/1 [==============================] - 0s 31ms/step


In [148]:
sample_prediction

array([[1]])

# Save the model

In [140]:
model.save("./assets/movieSentiAnalysisV-3.h5")

In [141]:
from tensorflow.keras.models import load_model

model_json = model.to_json()

# Save the JSON string to a file
with open("./assets/my_model-V3.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("./assets/my_model_weights-V3.h5")

In [142]:
from tensorflow.keras.models import load_model

In [143]:
rnn = load_model("./assets/movieSentiAnalysisV-3.h5")

In [104]:
print(data['review'][9],'\n')
print(data['sentiment'][9])

If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!! 

positive


In [165]:
text = """
    This movie is really fucking distinguish movie. people really do not like this type of movies.
"""

In [166]:
#preprocessing 
processed_text = remove_stop_words(text)
#one hot encoding
onehot_sent = [one_hot(word, voc_size) for word in processed_text]
#embedding
embedd_docs = pad_sequences(onehot_sent, padding='pre', maxlen=sent_length)
#convert into numpy array
X_sample = np.array(embedd_docs)

In [167]:
processed_text

['thi movi is realli fuck distinguish movi peopl realli do not like thi type of movi']

In [168]:
sample_prediction = (rnn.predict(X_sample) > 0.9).astype("int32")
sample_prediction

1/1 [==============================] - 0s 24ms/step


array([[1]])